# Including $ T_2 $ relaxation effects

In this notebook we demonstrate how to include $ T_2 $ relaxation to the tissue models. We recommend first familiarizing yourself with the basic functionality of MICROtool before using this functionality.


## Setting up the dmipy model


In [3]:
from dmipy.signal_models.gaussian_models import G1Ball

from microtool.dmipy import make_microtool_tissue_model
from microtool.tissue_model import RelaxedMultiTissueModel

# setting up a simple model in dmipy, where lambda iso represents the diffusivity
simple_ball_dmipy = G1Ball(lambda_iso=1.7e-9)

# converting it to a microtool compatible model
simple_ball_microtool = make_microtool_tissue_model(simple_ball_dmipy)

# adding relaxation
ball_with_relaxation = RelaxedMultiTissueModel(models=[simple_ball_microtool],volume_fractions= [1.0], relaxation_times=[0.020])

## Setting up the acquisition scheme

In [4]:
import numpy as np
from microtool.acquisition_scheme import DiffusionAcquisitionScheme
from microtool.gradient_sampling import sample_uniform_half_sphere

M = 10
N = 30

b_vals = np.concatenate([np.repeat(0, M), np.repeat(1000, M), np.repeat(3000, M)])
pulse_widths = np.concatenate([np.repeat(0.019, M), np.repeat(0.016, M), np.repeat(0.007, M)])
pulse_intervals = np.concatenate([np.repeat(0.030, M), np.repeat(0.027, M), np.repeat(0.020, M)])

directions = sample_uniform_half_sphere(N)
initial_scheme = DiffusionAcquisitionScheme.from_bvals(b_values=b_vals, b_vectors=directions, pulse_widths=pulse_widths,
                                                       pulse_intervals=pulse_intervals)

initial_scheme.fix_b0_measurements()
initial_scheme["EchoTime"].fixed = True
print(initial_scheme)

Acquisition scheme with 30 measurements and 6 scalar parameters. 
total number of optimized parameters is 60:
  DiffusionPulseMagnitude [mT/mm] in (0.0, 5000.0)    DiffusionGradientAnglePhi [rad] (fixed)    DiffusionGradientAngleTheta [rad] (fixed)    DiffusionPulseWidth [s] in (0.001, 0.1)    DiffusionPulseInterval [s] in (0.001, 0.1)    EchoTime [s] (fixed)
--------------------------------------------------  -----------------------------------------  -------------------------------------------  -----------------------------------------  --------------------------------------------  ----------------------
                                         0                                         -0.262465                                      1.5708                                        0.019                                         0.03                0.0349629
                                         0                                          0.610784                                      1.02

Now that we have the acquisition scheme and tissue model defined we test the generation of signal. As we can see we get reduced signal because of $T_2$ decay with respect to the model presented in the basic_functionality notebook

In [5]:
ball_with_relaxation(initial_scheme)

array([0.17409669, 0.17409669, 0.17409669, 0.17409669, 0.17409669,
       0.17409669, 0.17409669, 0.17409669, 0.17409669, 0.17409669,
       0.02625138, 0.02625138, 0.02625138, 0.02625138, 0.02625138,
       0.02625138, 0.02625138, 0.02625138, 0.02625138, 0.02625138,
       0.00075664, 0.00075664, 0.00075664, 0.00075664, 0.00075664,
       0.00075664, 0.00075664, 0.00075664, 0.00075664, 0.00075664])